<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# Dataframe Practice

In [48]:
sc.stop()
import pyspark as ps    # for the pyspark suite
from pyspark.sql import SQLContext

In [49]:
sc = ps.SparkContext('local[*]')
sqlContext = SQLContext(sc)

## Load the titanic dataset

In [51]:
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true',
            inferschema='true').load('../../../../resource-datasets/titanic/titanic_clean.csv')

## Print the schema

In [ ]:
df.printSchema()

## Create a temporary table view for SQL queries

In [52]:
df.createOrReplaceTempView("titanic")

## In all following questions, use both, dataframe queries and SQL queries

## Show the top of the dataframe.

In [53]:
df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+-------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+-------+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|   7.25|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|71.2833|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|  7.925|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|   53.1|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|   8.05|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+-------+--------+
only showing top 5 rows



In [54]:
sqlContext.sql("SELECT * FROM titanic LIMIT 5").show()

+-----------+--------+------+--------------------+------+----+-----+-----+-------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+-------+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|   7.25|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|71.2833|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|  7.925|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|   53.1|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|   8.05|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+-------+--------+



## Determine the number of observations

In [55]:
df.count()

712

In [58]:
sqlContext.sql("SELECT COUNT(*) FROM titanic").show()

+--------+
|count(1)|
+--------+
|     712|
+--------+



## Determine the correlation between `Fare` and `Age`

In [61]:
df.corr('Fare','Age')

0.09314251789411518

In [130]:
sql = """
SELECT CORR(Fare, Age)
FROM titanic
"""
sqlContext.sql(sql).show(15)

+-------------------+
|    corr(Fare, Age)|
+-------------------+
|0.09314251789411518|
+-------------------+



## Determine the minimal and maximal ages

In [86]:
df.describe().show(truncate=8)

+-------+-----------+--------+--------+--------+------+--------+--------+--------+--------+--------+
|summary|PassengerId|Survived|  Pclass|    Name|   Sex|     Age|   SibSp|   Parch|    Fare|Embarked|
+-------+-----------+--------+--------+--------+------+--------+--------+--------+--------+--------+
|  count|        712|     712|     712|     712|   712|     712|     712|     712|     712|     712|
|   mean|   448.5...|0.404...|2.240...|    null|  null|29.64...|0.514...|0.432...|34.56...|    null|
| stddev|   258.6...|0.491...|0.836...|    null|  null|14.49...|0.930...|0.854...|52.93...|    null|
|    min|          1|       0|       1|"Ande...|female|    0.42|       0|       0|     0.0|       C|
|    max|        891|       1|       3|van B...|  male|    80.0|       5|       6|512.3292|       S|
+-------+-----------+--------+--------+--------+------+--------+--------+--------+--------+--------+



In [88]:
sqlContext.sql("SELECT MIN(Age), MAX(Age) FROM titanic").show()

+--------+--------+
|min(Age)|max(Age)|
+--------+--------+
|    0.42|    80.0|
+--------+--------+



## Determine the number of passengers with `Age` below 40

In [89]:
df.filter("Age < 40").count()

550

In [93]:
sqlContext.sql("SELECT COUNT(PassengerId) FROM titanic WHERE Age < 40").show()

+------------------+
|count(PassengerId)|
+------------------+
|               550|
+------------------+



## Determine the number of survivors and non-survivors in the under 40 age group. Sort by `Survived`.

In [99]:
df.filter("Age < 40").groupby('Survived').count().sort('Survived', ascending=True).show()

+--------+-----+
|Survived|count|
+--------+-----+
|       0|  322|
|       1|  228|
+--------+-----+



In [129]:
#
sql = '''
SELECT Survived, COUNT(PassengerId)
FROM titanic
WHERE Age < 40
GROUP BY Survived
ORDER BY Survived ASC
'''

sqlContext.sql(sql).show()

+--------+------------------+
|Survived|count(PassengerId)|
+--------+------------------+
|       0|               322|
|       1|               228|
+--------+------------------+

